<a href="https://colab.research.google.com/github/ali1903/AI/blob/main/ANN_keras_Train_Test_Final_FeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Drive Baglantisi**


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
%cd /content/gdrive/My Drive/Colab Notebooks/Artificial Neural Network
%ls

/content/gdrive/My Drive/Colab Notebooks/Artificial Neural Network
ANN_keras.ipynb                                    deneme.h5
ANN_keras-Train-Test-Final-FeatureSelection.ipynb  diabetes.csv
ANN_keras-Train-Test-Final.ipynb                   Final_Veriseti.csv
ANN_keras-Train-Test.ipynb                         Numara_AdSoyad.docx
ANN_scikit.ipynb


**Verileri Oluştur**

In [10]:
#Gereken kutuphaneler yuklendi
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split  #Rastgele Train Test Bolmek icin
from sklearn.preprocessing import MinMaxScaler        #Min-Max normalization yapmak icin
import numpy

numpy.random.seed(7)
#veriseti yukleme
dataset=numpy.loadtxt("Final_Veriseti.csv", delimiter=",")

#datalar 0:16 arasi featurelar alindi
datalar=dataset[:,0:16]
datalar=MinMaxScaler().fit_transform(datalar)
#Veriler [0 1] araligina normalize edildi

etiketler=dataset[:,16]
#etiketler 16.indis class (outcome) alindi
etiketler=np_utils.to_categorical(etiketler, 7)
# Class isimleri 0,1,2,3,...6 iken 7 cikis icin ayarlandi 
# 0= [1,0,0,0,0,0,0], 1=[0,1,0,0,0,0,0], 2=[0,0,1,0,0,0,0] ...

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(datalar, etiketler, test_size=0.2, random_state=0)
#X_train: Train Datalari
#X_test:  Test Dataları
#Y_train: Train Etiketleri
#Y_test:  Test Etiketleri

In [12]:
print(etiketler.shape)
print(etiketler[13600,:])
#print(datalar.shape)
#print(datalar)

(13611, 7)
[0. 0. 0. 0. 0. 0. 1.]


In [14]:
#Sequential Feature Selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

sfs = SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3),
                    k_features=8, #Kac feature secilecek
                    forward=True, #İleri yonlu ise true, geri yonlu ise false
                    floating=False,
                    scoring='accuracy') #cv=1
X_new=sfs.fit_transform(X_train, Y_train)
print(X_train.shape)
print(X_new.shape)

(10888, 16)
(10888, 8)


In [15]:
print(sfs.subsets_[5])
#'feature_idx' icerisindeki featurelar secilen featurelar

{'feature_idx': (1, 10, 12, 13, 15), 'cv_scores': array([0.91414141, 0.90312213, 0.91781451, 0.90996785, 0.91134589]), 'avg_score': 0.911278357551377, 'feature_names': ('1', '10', '12', '13', '15')}


In [ ]:
#PCA: Principle Component Analysis
from sklearn.decomposition import PCA

pca = PCA(n_components=10) #n_components: Kac feature a donusturulecek
fit = pca.fit(X_train)
X_train_new = fit.transform(X_train)
X_test_new =fit.transform(X_test)
# summarize components
#print("Explained Variance: %s" % fit.explained_variance_ratio_)
#print(fit.components_)

print(X_train.shape)
print(X_train_new.shape)

(10888, 16)
(10888, 10)


**MLP Yapisi Kurulumu**

In [36]:
model=Sequential()
#8 feature girisli oldugu icin input_dim=16 secildi
model.add(Dense(140, input_dim=16, activation='linear')) # Hidden Layer 1 (25 noron)
#hidden layer 2 ye input_dim yazilmaz.
model.add(Dense(100, activation='tanh'))              # Hidden Layer 2  (100 noron)
model.add(Dense(220, activation='tanh'))              # Hidden Layer 2  (100 noron)

#Cikis 7 adet oldugu icin oldugu icin Dense 7 output noron baglandi
model.add(Dense(7, activation='softmax'))            #Output Layer
#activation=sigmoid, relu, linear, tanh, softmax,elu,swish,softplus

#Modeli Derleme
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#loss=KLD, mae, mse, mean_squared_error, cosine_proximity, binary_crossentropy, poisson, categorical_crossentropy
#optimizer: adam, RMSprop, SGD

#batch_size: Egitim sirasinda verisetinden ornekler kacar kacar alinacak
model.fit(X_train, Y_train, epochs=480, batch_size=100, verbose=1)
#verbose=0 -> Eğitimi adim adim gostermez sadece son epoch getirir
basarim=model.evaluate(X_train, Y_train)

print("\n Train %s : %.2f%% " % (model.metrics_names[1], basarim[1]*100))


Epoch 1/480
109/109 [==============================] - 1s 3ms/step - loss: 0.3393 - accuracy: 0.5159
Epoch 2/480
109/109 [==============================] - 0s 3ms/step - loss: 0.1244 - accuracy: 0.8743
Epoch 3/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0895 - accuracy: 0.8820
Epoch 4/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0731 - accuracy: 0.9036
Epoch 5/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0670 - accuracy: 0.9131
Epoch 6/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0660 - accuracy: 0.9066
Epoch 7/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0630 - accuracy: 0.9156
Epoch 8/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 0.9127
Epoch 9/480
109/109 [==============================] - 0s 4ms/step - loss: 0.0622 - accuracy: 0.9123
Epoch 10/480
109/109 [==============================] - 0s 3ms/step - loss: 0.0631 - accura

In [37]:
model.summary()
#model.get_config()
#from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='model.png')

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 140)               2380      
_________________________________________________________________
dense_58 (Dense)             (None, 100)               14100     
_________________________________________________________________
dense_59 (Dense)             (None, 220)               22220     
_________________________________________________________________
dense_60 (Dense)             (None, 7)                 1547      
Total params: 40,247
Trainable params: 40,247
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.save("97.73(eskiVeriseti).h5")

**Confusion Matrix**

In [38]:
from sklearn.metrics import classification_report,confusion_matrix

_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('>Test Accuracy: %.2f%%' % (test_acc * 100.0))


#yPredicted = model.predict(X_test)           #Direk cikista alinan degerleri verir
#yPredicted = model.predict_proba(X_test)     #Olasiliksal olarak cikti verir
yPredicted = model.predict_classes(X_test)
Y_Test_actual=numpy.argmax(Y_test,axis=1)

print(classification_report(y_true=Y_Test_actual, y_pred=yPredicted))
print(confusion_matrix(y_true=Y_Test_actual, y_pred=yPredicted))


>Test Accuracy: 92.66%
              precision    recall  f1-score   support

           0       0.95      0.96      0.95       376
           1       0.95      0.87      0.91       255
           2       1.00      1.00      1.00        92
           3       0.93      0.95      0.94       350
           4       0.97      0.95      0.96       390
           5       0.87      0.88      0.87       525
           6       0.92      0.93      0.92       735

    accuracy                           0.93      2723
   macro avg       0.94      0.93      0.94      2723
weighted avg       0.93      0.93      0.93      2723

[[360   1   0   1   0  12   2]
 [  5 222   0  19   3   6   0]
 [  0   0  92   0   0   0   0]
 [  1   9   0 334   3   3   0]
 [  0   1   0   7 370  10   2]
 [  3   1   0   0   4 462  55]
 [ 11   0   0   0   1  40 683]]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
